# Select asteroid comparisons; JPL Horizon vs OpenOrb, OrbFit

## Assumptions/ documentation:

* Values for orbit definition taken from JPL Horizons elements
* Observing Location: I11, Gemini South
* Integration Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:
* OrbFit 60 arcsecond bug
* OrbFit Covariance matrix effect/ workaround
* Documenting functions (work in progress)
* Workaround for np.array() of RA/DEC w/ deg units not being compatible with np.array() of Mags in mag units
* Find_Orb takes state vectors, also not available in COM

## Info Pointers
* table color-coding: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
* Keep sign separate from columns: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_fwf.html, https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#csv-text-files

* OrbFit questions via email thread w/ Federica
* Find_Orb questions in Slack

# Clean code

In [1]:
%matplotlib inline

import sys
import re
import numpy as np
import pandas as pd
import jinja2
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import QTable
from astropy.io import ascii
import astropy as ap
import matplotlib.pyplot as plt
from astroquery.jplhorizons import Horizons
import pyoorb as oo
oo.pyoorb.oorb_init()

0

In [2]:
# Initalization variables: times (UTC), location, object (find a way to generalize objects)
obj_id = '202930'
start = '2010-01-01T00:00:00'
stop = '2020-01-01T00:00:01'
obs = 'I11@399'

## JPL Horizons

In [25]:
# calculates elements and ephemerides for object
def get_ephem_jpl(obj_id, start, stop, obs):
    '''
    Parameters
    ----------
    obj_id : `str`
        Number of asteroid, defined by MPC.
    
    start : `str`
        Beginning time of integration, UTC.
        
    stop : `str`
        End time of integration, UTC.
        
    obs : `str`
        Observatory code.
        
    Returns
    -------
    el_jpl : `~numpy.ndarray` (N, 12)
        Orbital elements as returned by jpl. 
            
    coord_jpl : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by JPL, units deg
            RA : first column of coord_jpl
            DEC : second column of coord_jpl
    '''
    
    epochs = ap.time.Time(start).jd
    
    el_obj = Horizons(id=obj_id, location= '500@10',
               epochs=epochs)
    el_jpl = el_obj.elements()
    print("JPL elements done")
    ephem_obj = Horizons(id= obj_id, location= obs,
               epochs={'start': start, 'stop':stop,
                      'step':'1d'})
    ephem_jpl = ephem_obj.ephemerides()
    print("JPL ephems done")

    coord_jpl = np.array([ephem_jpl['RA'], ephem_jpl['DEC']]) * u.deg
    mag_jpl = np.array([ephem_jpl['V']]) * u.mag
    
    return el_jpl, coord_jpl, mag_jpl

## OpenOrb/ pyoorb

In [26]:
# Reorganizes JPL Horizons elements output into pyoorb-acceptable input. (Expand for multiple orbits next?)
def pyoorb_input(orbits, epoch):
    '''
    Parameters
    ----------
    orbits : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    epoch : `~numpy.ndarray` (3652, 2)
        Constrained to cometary format.
        
    Returns
    -------
    new_array : `~numpy.ndarray` (N, 12)
        Orbits formatted in the format expected by PYOORB. 
            orbit_id : index of input orbits
            elements x6: orbital elements of propagated orbits
            orbit_type : orbit type
            epoch_mjd : epoch of the propagate orbit
            time_scale : time scale of output epochs
            H/M1 : absolute magnitude
            G/K1 : photometric slope parameter
    '''
    
    temp = orbits.copy()
    temp = temp.as_array().data
    if temp.shape == (6,):
        num_orbits = 1
    else:
        num_orbits = temp.shape[0]
        
    for i in range(num_orbits):
        ids = i
        orbit_type = 2
        time_scale = 3
        
    # elements x6
    q = temp[0][6]
    e = temp[0][5]
    incl = np.deg2rad(temp[0][7])
    longnode = np.deg2rad(temp[0][8])
    argper = np.deg2rad(temp[0][9])
    peri_epoch = ap.time.Time(temp[0][10], format='jd').mjd

    mag = temp[0][3]
    slope = temp[0][4]
    
    if num_orbits > 1:
        new_array = np.array(
            np.array([
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]), 
            dtype=np.double, 
            order='F')
    else:
        new_array = np.array(
            [[
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]], 
            dtype=np.double,
            order='F')
    
    return new_array

In [27]:
# Calculates ephemerides using PYOORB
def get_ephem_OpenOrb(el_jpl, start, stop):
    '''
    Parameters
    ----------
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
   start : `str`
        Beginning time of integration, UTC.
        
    stop : `str`
        End time of integration, UTC.
        
    Returns
    ------- 
    coord_OpenOrb : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by PYOORB, units deg
            RA : first column of coord_OpenOrb
            DEC : second column of coord_OpenOrb 
    '''
    
    # time conversions, epochs for pyoorb to work
    element_time_pyoorb = ap.time.Time(start).mjd
    start_pyoorb = ap.time.Time(start).mjd
    stop_pyoorb = ap.time.Time(stop).mjd
    peri_time = ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd
    
    #conversion and implementation
    pyoorb_formatted = pyoorb_input(el_jpl, start_pyoorb)
    print('JPL elements ready for OpenOrb use')
    t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
    mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
    epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
    ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=pyoorb_formatted,
                                         in_obscode='I11',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')
    print("OpenOrb ephemerides done")

    coord_OpenOrb = np.array([ephem_pyoorb[0][:,1],ephem_pyoorb[0][:,2]]) * u.deg
    mag_OpenOrb = np.array([ephem_pyoorb[0][:,9]]) * u.mag
    
    return coord_OpenOrb, mag_OpenOrb

## OrbFit

In [28]:
# Creates .eq1 file in epoch folder for OrbFit
def eq1file(el_jpl):
    '''
    Parameters
    ----------
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
        
    Returns
    ------- 
    temp.eq1: temporary text file to run OrbFit
    '''
    
    with open(f"2019OK/epoch/temp.eq1", "w") as fp:
        fp.write("format  = 'OEF2.0'       ! file format\n")
        fp.write("rectype = 'ML'           ! record type (1L/ML)\n")
        fp.write("refsys  = ECLM J2000     ! default reference system\n")
        fp.write("END_OF_HEADER\n")
        fp.write(f"temp\n")
        fp.write("! Cometary elements: q, e, i, long. node, arg. peric., pericenter time\n")
        fp.write(" COM   %.15E  %.15f   %.15f  %.15f  %.15f   %.15f\n" % (el_jpl['q'][0], el_jpl['e'][0], el_jpl['incl'][0], el_jpl['Omega'][0], el_jpl['w'][0], ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd))
        fp.write(" MJD     %.9f TDT\n" % (ap.time.Time(el_jpl['datetime_jd'][0], format='jd').mjd))
        fp.write(" MAG  %.3f  %.3f\n" % (el_jpl['H'][0], el_jpl['G'][0]))
        fp.write("! Non-grav parameters: model used, actual number in use, dimension\n")
        fp.write(" LSP   0  0    6\n")
        fp.write("! RMS    1.68232E-08   5.88900E-08   8.22688E-08   5.34469E-08   7.56890E-08   7.13398E-06\n")
        fp.write("! EIG   1.07579E-08   5.31009E-08   5.91864E-08   7.57356E-08   7.86169E-08   1.27486E-07\n")
        fp.write("! WEA   0.08651  -0.02289  -0.24412  -0.03357  -0.02078  -0.96480\n")
        fp.write(" COV   2.830210138102556E-16  3.543122024213312E-16 -2.603292682702056E-16\n")
        fp.write(" COV  -4.992042214900484E-18 -1.052690180196314E-18 -7.873861865190710E-14\n")
        fp.write(" COV   3.468027286557968E-15  6.878077752471183E-17  1.886511729787680E-17\n")
        fp.write(" COV   6.689670038864485E-17  1.808279351538482E-14  6.768149177265766E-15\n")
        fp.write(" COV   7.159243161286040E-17  1.248926483233068E-16  1.357728291186093E-13\n")
        fp.write(" COV   2.856568560436748E-15  2.588049637167598E-16  2.529981071526617E-14\n")
        fp.write(" COV   5.728829671236270E-15  1.056596023015451E-14  5.089368128905899E-11\n")
        fp.write(" NOR   8.462990106959648E+15 -9.345934921051774E+14  6.961302078833404E+13\n")
        fp.write(" NOR  -9.766026206616650E+13 -9.148695418092123E+12  1.329006055003970E+13\n")
        fp.write(" NOR   3.921580648140324E+14 -8.566206317904612E+12  1.006265833999790E+13\n")
        fp.write(" NOR  -2.128841368531985E+12 -1.566971456817283E+12  1.567202493495569E+14\n")
        fp.write(" NOR  -7.910041612493922E+11 -2.702958007388599E+12 -3.063965034542373E+11\n")
        fp.write(" NOR   3.541407046591562E+14 -1.551670529664669E+13 -3.253830675316872E+11\n")
        fp.write(" NOR   1.754031538722264E+14 -3.488851624201696E+10  4.175326401599722E+10\n")
    return

In [29]:
# Writes .fop file for OrbFit
def fopfile():
    '''
    Parameters
    ----------
    
    Returns
    ------- 
    temp.fop: temporary text file to run OrbFit
    '''
    
    with open(f"2019OK/temp.fop", "w") as fp:
        fp.write("! input file for fitobs\n")
        fp.write("fitobs.\n")
        fp.write("! first arc")
        fp.write(f"        .astna0='temp'           ! full name, first arc\n")
        fp.write("        .obsdir0='mpcobs/'         ! directory of observs. file, first arc\n")
        fp.write(f"        .elefi0='epoch/temp.eq1' ! first arc elements file\n")
        fp.write("\n")
        fp.write("! second arc\n")
        fp.write("!        .astnap=''            ! full name, second arc\n")
        fp.write("!        .obsdirp='mpcobs'     ! directory of observs. file, second arc\n")
        fp.write("! bizarre  control;\n")
        fp.write("        .ecclim=     1.9999d0    ! max eccentricity for non bizarre orbit\n")
        fp.write("        .samin=      0.3d0       ! min a for non bizarre orbit\n")
        fp.write("        .samax=      2000.d0     ! max a for non bizarre orbit\n")
        fp.write("        .phmin=      0.001d0     ! min q for non bizarre orbit\n")
        fp.write("        .ahmax=     4000.d0      ! max Q for non bizarre orbit\n")
        fp.write("        .error_model='fcct14'     ! error model\n")
        fp.write("propag.\n")
        fp.write("        .iast=17            ! 0=no asteroids with mass, n=no. of massive asteroids (def=0)\n")
        fp.write("        .filbe='AST17'      ! name of the asteroid ephemerides file (def='CPV')\n")
        fp.write("        .npoint=600         ! minimum number of data points for a deep close appr (def=100)\n")
        fp.write("        .dmea=0.2d0         ! min. distance for control of close-app. to the Earth only (def=0.1)\n")
        fp.write("        .dter=0.05d0        ! min. distance for control of close-app.\n")
        fp.write("                            ! to terrestrial planets (MVM)(def=0.1)\n")
        fp.write("        .yark_exp=2.d0      ! A2/r^yark_exp model (def=2)\n")
        fp.write("        .ngr_opt=.TRUE.     ! read options for non-gravitational perturbations from the option file\n")
        fp.write("        .irel=2             ! 0=newtonian 1=gen. relativity, sun 2=gen. rel. all planets\n")
        fp.write("                            !          (def=0, 1 for NEA, 2 for radar)\n")
        fp.write("        .iaber=2            ! aberration 0=no 1=yes 2=(def=1)\n")
        fp.write("        .ilun=1             ! 0=no moon 1= yes (def=0, 1 for NEA)\n")
        fp.write("        .iyark=3            ! 0=no Yarkovsky, 1=Yark diurnal, 2=Yark seasonal\n")
        fp.write("                            !    3=secular nongravitational perturbations (including Yark) (def=0)\n")
        fp.write("        .ipa2m=0           ! 0=no drpa2m, 1=yes spherical direct radiation pressure (def=0)\n")
        fp.write("        .det_drp=2          ! how many parameters to solve: 0=none 1=drpa2m 2=dadt 3=both (def=0)\n")
        fp.write("        .det_outgas=0       ! det outgassing for comets\n")
        fp.write("\n")
        fp.write("difcor.\n")
        fp.write("\n")
        fp.write("IERS.\n")
        fp.write("        .extrapolation=.T. ! extrapolation of Earth rotation\n")
        fp.write("\n")
        fp.write("reject.\n")
        fp.write("        .rejopp=.false.    ! reject entire opposition\n")
    return

In [30]:
# Writes file to bypass OrbFit's interactive menu
def astfile(start, stop, obs):
    '''
    Parameters
    ----------
    start : `str`
        Beginning time of integration, UTC.
        
    stop : `str`
        End time of integration, UTC.
        
    obs : `str`
        Observatory code.
        
    Returns
    ------- 
    ast.inp: text file required to bypass OrbFit interactive menu
    '''
    
    with open("2019OK/ast.inp", "w") as fp:
        fp.write(f"temp\n")
        fp.write("6\n")
        fp.write("6\n")
        fp.write(f"{ap.time.Time(start).mjd}\n")
        fp.write(f"{ap.time.Time(stop).mjd}\n")
        fp.write("1\n")
        fp.write(f"{obs}\n")
        fp.write("0\n")
    return

In [31]:
# Calculates ephemerides using OrbFit
def get_ephem_OrbFit(obj_id, el_jpl, start, stop, obs):
    '''
    Parameters
    ----------
    obj_id : `str`
        Number of asteroid, defined by MPC.
        
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    
    start : `str`
        Beginning time of integration, UTC.
        
    stop : `str`
        End time of integration, UTC.
        
    obs : `str`
        Observatory code.
        
    Returns
    -------
    coord_OrbFit : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by OpenOrb, units deg
            RA : first column of coord_OrbFit
            DEC : second column of coord_OrbFit  
    '''
    
    eq1file(el_jpl)
    fopfile()
    astfile(start, stop, obs)
    print("Orbfit prep done")

    ! (cd 2019OK && ./fitobs.x < ast.inp) # > /dev/null inside parenth when fixed
    df = pd.read_fwf(f'2019OK/temp.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55),(56, 61),(62,67)])
    
    df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
    df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
    df.loc[df[4] == '-', "DEC"] *= -1
    df['Mag'] = df[8]
    
    coord_OrbFit = np.array([df["RA"], df["DEC"]]) * u.deg
    mag_OrbFit = np.array([df['Mag']]) * u.mag
    print("OrbFit done")
    return coord_OrbFit, mag_OrbFit

## Outputs and comparison

In [32]:
# Takes in object id and calls all integrators
def get_ephems(obj_id, start, stop, obs):
    '''
    Parameters
    ----------
    obj_id : `str`
        Number of asteroid, defined by MPC.
        
    start : `str`
        Beginning time of integration, UTC.
        
    stop : `str`
        End time of integration, UTC.
        
    obs : `str`
        Observatory code.
        
    Returns
    -------
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
        
    coord_jpl : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by JPL, units deg
            RA : first column of coord_jpl
            DEC : second column of coord_jpl
        
    coord_OpenOrb : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by PYOORB, units deg
            RA : first column of coord_OpenOrb
            DEC : second column of coord_OpenOrb
            
    coord_OrbFit : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by OpenOrb, units deg
            RA : first column of coord_OrbFit
            DEC : second column of coord_OrbFit  
    '''
    
    el_jpl, coord_jpl, mag_jpl = get_ephem_jpl(obj_id, start, stop, obs)
    coord_OpenOrb, mag_OpenOrb = get_ephem_OpenOrb(el_jpl, start, stop)
    coord_OrbFit, mag_OrbFit = get_ephem_OrbFit(obj_id, el_jpl, start, stop, obs)
    
    return el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit


# attempt to refactor ephems for list of objects

def get_ephems2(obj_ids, start, stop, obs):
    collector_el_jpl = 
    collector_coord_jpl = np.array()
    collector_mag_jpl = np.array()
    collector_coord_OpenOrb = np.array()
    collector_mag_OrbFit = np.array()
    collector_coord_OrbFit = np.array()
    collector_mag_OrbFit = np.array()
    
    for i in rangelen(obj_ids):
        np.append(collector_, appended_array, axis=0)
        
        el_jpl, coord_jpl, mag_jpl = get_ephem_jpl(obj_ids[i], start, stop, obs)
        coord_OpenOrb, mag_OpenOrb = get_ephem_OpenOrb(el_jpl, start, stop)
        coord_OrbFit, mag_OrbFit = get_ephem_OrbFit(obj_id[i], el_jpl, start, stop, obs)
    
    return

## Difference metric

In [33]:
# Maximum circle difference function. Calculates great circle difference and other difference metrics (future)
def max_diff(param1, param2):
    '''
    Parameters
    ----------
    param1: `~numpy.ndarray` (N, 2)
        Parameters from 1st integrator.
    param2: `~numpy.ndarray` (N, 2)
        Parameters from 2nd integrator. 
    
    Returns
    -------
    diff_matrix: `~numpy.ndarray` (N, 2)
        Separation between param1 and param2, units arcsec, other difference metrics added soon. 
    '''
    
    #coordinates
    coord1 = SkyCoord(param1[0], param1[1], frame='icrs', unit="deg")
    coord2 = SkyCoord(param2[0], param2[1], frame='icrs', unit="deg")
    sep = coord1.separation(coord2)
    
    # magnitudes
    #mag1 = ephem1['V']
    #mag2 = ephem2['V']
    
    #magnitude = mag2 - mag1
    
    # matrix
    # degs, mins, secs; extract secs
    diff_matrix = sep.arcsec
        
    return diff_matrix.flatten()

In [34]:
# Calculates basic statistics of data
def table_stats(array):
    '''
    Parameters
    ----------
    array: `~numpy.ndarray` (1, N)
        Array of numbers.
    
    Returns
    -------
    statistics: `~numpy.ndarray` (1, 3)
        Array with median, mean, maximum.
    '''
    
    median = np.median(array)
    mean = np.mean(array)
    maximum = np.max(array)
    statistics = np.array([median, mean, maximum])
    return statistics

# Work-in-progress

## Graphical Representation

In [13]:
def get_ephem_FindOrb():
    !./fo '2019OK/temp.eq1' -ephem
    
    return coord_FindOrb

In [35]:
# Creates color coding map for tables
def color_map(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: green']
        elif val < 0.2:
            style = ['background-color: yellow']
        elif val < 0.6:
            style = ['background-color: orange']
        else:
            style = ['background-color: red']
        
        ret += style
    return ret

In [36]:
def table_gen(integrator1, integrator2):
    
    statistics1 = table_stats(integrator1)
    statistics2 = table_stats(integrator2)

    total_stats = np.array([statistics1, statistics2]) * u.arcsec

    table = pd.DataFrame(data= total_stats.transpose(), columns=['OpenOrb', 'OrbFit'])
    table = table.rename(index={0: "Median", 1: "Mean", 2: "Maximum"})
    table = table.rename_axis("Metric", axis="columns")
    table = table.style.apply(color_map)
    return table

In [37]:
def outputs(baseline, integrator1, integrator2):
    
    great_circle_diff1 = max_diff(baseline, integrator1)
    print("Precision of OpenOrb w/r/t JPL:", great_circle_diff1)

    print()
    great_circle_diff2 = max_diff(baseline, integrator2)
    print("Precision of Orbfit w/r/t  JPL:", great_circle_diff2)
    
    return great_circle_diff1, great_circle_diff2, table_gen(great_circle_diff1, great_circle_diff2)

In [38]:
# table generation, but for the entire integrator- get stats for all of the asteroids in sum
def output(asteroid1, asteroid2, asteroid3):
    
    max_out1 = np.max(asteroid1)
    max_out2 = np.max(asteroid2)
    max_out3 = np.max(asteroid3)
    
    total_max = np.array([max_out1, max_out2, max_out3]) * u.arcsec
    
    table = pd.DataFrame(data= total_max.transpose(), columns=['OpenOrb', 'OrbFit'])
    table = table.rename(index={0: "NEO asteroid", 1: "Minimoon", 2: "Main belt", 3: "Trojan", 4: "KBO"})
    table = table.rename_axis("Asteroid Type", axis="columns")
    table = table.style.apply(color_map)
    
    return table

### Key:
value <= 0.05" -> green

value <= 0.2" -> yellow

value <= 0.6" -> orange

greater values -> red

# Prep Zone

Let's adopt these:

value <= 0.05" -> green (good)

value <= 0.2" -> yellow (ok)

value <= 0.6" -> orange (not good but may work)

value is red otherwise (bad)

In [18]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(obj_id, start, stop, obs)
great_circle_diff1, great_circle_diff2, table = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

1       53.91
2       32.62
3        6.69
4       36.05
        ...  
3648    56.16
3649     8.19
3650    15.92
3651    19.34
3652    18.44
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Precision of OpenOrb w/r/t JPL: [0.00785716 0.01113825 0.00619841 ... 0.1361494  0.12737095 0.1156872 ]

Precision of Orbfit w/r/t  JPL: [0.01339081 0.00844377 0.01432212 ... 0.0540432  0.06476943 0.08087179]


Metric,OpenOrb,OrbFit
Median,0.024776,0.020844
Mean,0.041732,0.024990
Maximum,0.156815,0.084155


## Tried it with Ceres no1, Vesta no4,  Lydia no110 (more MBA population)

In [ ]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems("1", start, stop, obs)
outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)

In [20]:
# this creates the .eq1 file in epoch folder
def eqfake(el_jpl):
    with open(f"2019OK/epoch/temp.eq1", "w") as fp:
        fp.write("format  = 'OEF2.0'       ! file format\n")
        fp.write("rectype = 'ML'           ! record type (1L/ML)\n")
        fp.write("refsys  = ECLM J2000     ! default reference system\n")
        fp.write("END_OF_HEADER\n")
        fp.write(f"temp\n")
        fp.write("! Cometary elements: q, e, i, long. node, arg. peric., pericenter time\n")
        fp.write(" COM   %.15E  %.15f   %.15f  %.15f  %.15f   %.15f\n" % (el_jpl['q'][0], el_jpl['e'][0], el_jpl['incl'][0], el_jpl['Omega'][0], el_jpl['w'][0], ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd))
        fp.write(" MJD     %.9f TDT\n" % (ap.time.Time(el_jpl['datetime_jd'][0], format='jd').mjd))
        fp.write(" MAG  %.3f  %.3f\n" % (el_jpl['H'][0], el_jpl['G'][0]))
        fp.write("! Non-grav parameters: model used, actual number in use, dimension\n")
        fp.write(" LSP   0  0    6\n")
        fp.write("! RMS    2.90115E-09   3.14395E-08   2.97594E-08   3.18456E-08   3.18245E-08   3.97334E-06\n")
        fp.write("! EIG   1.65099E-09   2.96887E-08   3.12090E-08   3.12852E-08   3.24854E-08   6.94718E-08\n")
        fp.write("! WEA   0.03427  -0.02611   0.01394   0.01518   0.04363  -0.99791\n")
        fp.write(" COV   8.416698768255657E-18 -1.465587306769135E-18  8.091167808126779E-19\n")
        fp.write(" COV   2.152561694154779E-18  3.420019040173357E-18 -9.467057278796147E-15\n")
        fp.write(" COV   9.884421842147777E-16 -1.792741974423255E-17  1.972862481202626E-17\n")
        fp.write(" COV   1.341097859619348E-17  5.786453296501320E-15  8.856234514608942E-16\n")
        fp.write(" COV  -2.929554547592711E-18  9.178242767249992E-18 -3.104286381493221E-15\n")
        fp.write(" COV   1.014142595955366E-15  3.701165572401731E-17 -3.258015713202228E-15\n")
        fp.write(" COV   1.012799173906613E-15 -9.526386951856685E-15  1.578739888043853E-11\n")
        fp.write(" NOR   3.664271831958311E+17 -7.502927727696384E+14  4.141676299982450E+14\n")
        fp.write(" NOR  -8.423042565980175E+13  8.466967531576468E+14  2.205813588463062E+14\n")
        fp.write(" NOR   1.016446496166562E+15  1.845984041020283E+13 -2.014255680369505E+13\n")
        fp.write(" NOR  -1.820131432400191E+13 -8.339814451077467E+11  1.130848390943001E+15\n")
        fp.write(" NOR   3.787636210520775E+12 -7.701868421488197E+12  4.600870723431448E+11\n")
        fp.write(" NOR   9.883397486155219E+14 -3.427571123289421E+13  1.408972033295217E+11\n")
        fp.write(" NOR   9.964802652131335E+14  1.107106042789595E+12  1.967085385584677E+11\n")   
        
    return

In [21]:
def get_ephem_OrbFit(obj_id, el_jpl, start, stop, obs):
    eqfake(el_jpl)
    fopfile()
    astfile(start, stop, obs)
    print("Orbfit prep done")

    ! (cd 2019OK && ./fitobs.x < ast.inp) # > /dev/null inside parenth when fixed
    df = pd.read_fwf(f'2019OK/temp.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55), (56, 61)])
    
    df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
    df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
    df.loc[df[4] == '-', "DEC"] *= -1
    
    RA_OrbFit = df["RA"]
    DEC_OrbFit = df["DEC"]
    coord_OrbFit = np.array([RA_OrbFit, DEC_OrbFit]) * u.deg
    print("OrbFit done")
    return coord_OrbFit

In [24]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(4, start, stop, obs)

great_circle_diff1 = max_diff(coord_jpl, coord_OpenOrb)
print("Precision of OpenOrb w/r/t JPL:", great_circle_diff1)

print()
great_circle_diff2 = max_diff(coord_jpl, coord_OrbFit)
print("Precision of Orbfit w/r/t  JPL:", great_circle_diff2)

table_gen(great_circle_diff1, great_circle_diff2)

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

1       42.35
2       21.05
3       14.15
4       25.89
        ...  
3648    27.55
3649    23.99
3650    21.56
3651    20.28
3652    20.18
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got 160.343025 deg

In [39]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(110, start, stop, obs)
gc1_lyd, gc2_lyd, table_lyd = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table_lyd

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

Metric,OpenOrb,OrbFit
Median,0.014360,0.020939
Mean,0.014114,0.024770
Maximum,0.034501,0.080562


## Try it with KBO (Eris, no 136199 ; Albion no15760)

In [40]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(136199, start, stop, obs)
gc1_er, gc2_er, table_er = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table_er

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

Metric,OpenOrb,OrbFit
Median,0.014260,0.015112
Mean,0.013767,0.015204
Maximum,0.025265,0.038391


In [41]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(15760, start, stop, obs)
gc1_alb, gc2_alb, table_alb = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table_alb

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

Metric,OpenOrb,OrbFit
Median,0.014073,0.014889
Mean,0.013605,0.015212
Maximum,0.025131,0.035735


## Try a PHA? (2018VP1, 101955, Apophis no99942) 

In [42]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems("2018 VP1", start, stop, obs)
gc1_vp1, gc2_vp1, table_vp1 = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table_vp1

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

Metric,OpenOrb,OrbFit
Median,0.021749,0.032386
Mean,0.686890,0.047743
Maximum,113.581918,16.163671


In [43]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(101955, start, stop, obs)
gc1_955, gc2_955, table_955 = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table_955

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

1       25.28
2        7.08
3       33.52
4       44.53
        ...  
3648     0.64
3649    20.43
3650    21.38
3651     3.59
3652    27.19
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


Metric,OpenOrb,OrbFit
Median,0.017896,0.056719
Mean,0.026510,0.074067
Maximum,0.180340,0.286108


In [44]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(99942, start, stop, obs)
gc1_42, gc2_42, table_42 = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table_42

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

1       12.30
2       52.51
3       55.72
4       22.16
        ...  
3648    35.74
3649     6.63
3650    17.99
3651    11.48
3652    48.79
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


Metric,OpenOrb,OrbFit
Median,0.047806,0.106786
Mean,0.057719,0.118999
Maximum,0.230640,0.371872


2020 AV2 (a Vatira), 2020 SO (Earth's mini moon), 367943 Duende, 99942 Apophis, 433 Eros, 
5261 Eureka (Mars Trojan), 

MBA: 887 Alinda, 6489 Golevka, 176014 Vedrana, 

Jupiter Trojans: 624 Hektor, 588 Achilles

Centaur: 10199 Chariklo

Interstellar object 'Oumuamua (A/2017 U1)

TNOs: 136472 Makemake (2005 FY9), 136199 Eris

Earth impactors: 2008 TC3

Funsies: 2020VT4


OO green, OF yellow, last orange; total red; total red ; total green; OO green, OF green, last yellow;
OO green, OF green, last yellow;
OO green, OF green, last yellow; total green, last row yellow; total green, last row yellow
OO green, OF green, last yellow; OO green, OF green, last yellow
total green
first row green, rest red
total green; total green
OO green, OF yellow, last red

Funsies: OO green, bottom yellow; OF green yellow red

In [45]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems("2008 TC3", start, stop, obs)
gcd1, gcd2, table = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
table

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

Metric,OpenOrb,OrbFit
Median,0.014004,0.050286
Mean,0.013516,0.070557
Maximum,0.025120,1.466696


In [46]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems("2020 VT4", start, stop, obs)
gcd1, gcd2, tabel = outputs(coord_jpl, coord_OpenOrb, coord_OrbFit)
tabel

JPL elements done
JPL ephems done
JPL elements ready for OpenOrb use
OpenOrb ephemerides done
Orbfit prep done
 Run name =
 JPL planetary ephemerides DE431
 number of perturbers =           17
 binary ephemerides file= AST17
 perturbing asteroids used: 16 major perturbing asteroids plus Pluto
 ngr_opt =  T
    rmodel: overriding the default for ngr_opt, now  T
 direct radiation pressure in the dynamical model =            0
 direct radiation pressure coefficient =    0.0000000000000000     
 Yarkovsky effect in the dynamical model =            3
 A2 Yarkovsky parameter =    0.0000000000000000     
 solve-for parameters selector (drpa2m, A2, or both) =            2
 outgassing model =            0
 A1 outgassing parameter =    0.0000000000000000     
 A2 outgassing parameter =    0.0000000000000000     
 A3 outgassing parameter =    0.0000000000000000     
 delay outgassing parameter =    0.0000000000000000     
 outgassing parameter to solve-for =            0
 determining parameter   

1       31.71
2       38.45
3       32.44
4       13.69
        ...  
3648    54.63
3649    19.03
3650    33.24
3651    37.21
3652    30.87
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


Metric,OpenOrb,OrbFit
Median,0.014723,0.043373
Mean,0.015325,0.051657
Maximum,0.113632,0.603225


In [47]:
gc1_mba = np.array([gc1_lyd])
max1_mba = np.max(gc1_mba)
gc2_mba = np.array([gc2_lyd])
max2_mba = np.max(gc2_mba)
max_mba = np.array([max1_mba, max2_mba])

gc1_kbo = np.array([gc1_er, gc1_alb])
max1_kbo = np.max(gc1_kbo)
gc2_kbo = np.array([gc2_er, gc2_alb])
max2_kbo = np.max(gc2_kbo)
max_kbo = np.array([max1_kbo, max2_kbo])

gc1_pha = np.array([gc1_vp1, gc1_955, gc1_42])
max1_pha = np.max([gc1_pha])
gc2_pha = np.array([gc2_vp1, gc2_955, gc2_42])
max2_pha = np.max([gc2_pha])
max_pha = np.array([max1_pha, max2_pha])


total_max = np.array([max_mba, max_kbo, max_pha])

In [ ]:
total_max

In [ ]:
pd.DataFrame(data= total_max,columns=['OpenOrb', 'OrbFit'])

In [ ]:
# Maximum on-sky difference relative to JPL (inconsistent and sparse sample size, beware)

In [48]:
table = pd.DataFrame(data= total_max, columns=['OpenOrb', 'OrbFit'])
table = table.rename(index={0: f"Main belt", 1: "KBO", 2: "PHA"})
table = table.rename_axis("Asteroid Type", axis="columns")
table = table.style.apply(color_map)
table

Asteroid Type,OpenOrb,OrbFit
Main belt,0.034501,0.080562
KBO,0.025265,0.038391
PHA,113.581918,16.163671


# Recipe

In [ ]:
'The value of pi is %10.8f slices (and %.3f).' % (3.14159, 456.2)

In [ ]:
'''
Gfortran version 4.8.6

Useful bits from docs:

JPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'

pyoorb:
orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
perihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART
eccentricity for COM or KEP, y (au) for CART
inclination (deg) for COM or KEP, z (au) for CART
longitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART
argument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT
epoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART
orbital element type; integer value: CART: 1, COM: 2, KEP: 3
epoch of the osculating elements (modified Julian date)
timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
absolute magnitude of the target
photometric slope parameter of the target


JPL TableColumns values to extract:
[0,1,2,3,7,8,23] = [targetname, datetime str, datettime jd, H, RA, DEC, V]

Want pyoorb.oorb_ephemeris_basic , use these indices for properties:
[0,1,2,9] = [mjd, RA (deg), DEC(deg), predicted apparent V-band mag]

example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation

OrbFit bug: 9 60.00 for seconds is wrong

18 May 2013 24.000 56430.999988    6 40  3.478  +24  9 60.00   8.6   6.7    7.933  -26.7  -41.1   58.2   8.4 190.0  2.5146  3.1868    1.0092   -0.0237    0.012"    0.010"  85.4

'''